In [6]:
# Your code here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import time 
from sklearn.feature_selection import SelectKBest, f_classif
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

In [7]:
import os
import pandas as pd

# Ruta base
BASE_PATH = "../data/processed/Excels/"

# Definir las rutas de los archivos de entrenamiento y prueba
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_sel_k4.xlsx"
]

TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_sel_k4.xlsx"
]

# Leer los datasets de entrenamiento
TRAIN_DATASETS = [pd.read_excel(os.path.join(BASE_PATH, path)) for path in TRAIN_PATHS]  # X_train 

# Leer los datasets de prueba
TEST_DATASETS = [pd.read_excel(os.path.join(BASE_PATH, path)) for path in TEST_PATHS]  # X_test

# Leer las etiquetas (target) de entrenamiento y prueba
y_train = pd.read_excel(os.path.join(BASE_PATH, "y_train.xlsx")).values.ravel()
y_test = pd.read_excel(os.path.join(BASE_PATH, "y_test.xlsx")).values.ravel()


In [8]:
# all_selected_features = {}
# model_results = []
# k = 6

# for index, dataset in enumerate(TRAIN_DATASETS):
#     print(f"Procesando dataset {index + 1}")

#     # Selección de características usando SelectKBest con f_classif (para clasificación)
#     selector = SelectKBest(f_classif, k=k)
#     X_train = selector.fit_transform(dataset, y_train)  # Ajuste y transformación para el conjunto de entrenamiento
#     X_test = selector.transform(TEST_DATASETS[index])  # Transformación para el conjunto de prueba

#     # Almacenar las características seleccionadas en el diccionario
#     selected_features = dataset.columns[selector.get_support()].tolist()  # Obtener los nombres de las características seleccionadas
#     all_selected_features[TRAIN_PATHS[index]] = selected_features  # Usar el nombre del archivo como clave

#     ### ENTRENAR MODELO ###

#     # Crear y entrenar el modelo de regresión logística (clasificación)
#     model = XGBClassifier(random_state = 95)  ### Asegúrate de ajustar los parámetros según sea necesario!!! 
#     model.fit(X_train, y_train)

#     # Predicciones en los datos de entrenamiento y prueba
#     y_train_pred = model.predict(X_train)
#     y_test_pred = model.predict(X_test)

#     # Calcular la precisión (accuracy) para los datos de entrenamiento
#     accuracy_train = accuracy_score(y_train, y_train_pred)
#     precision_train = precision_score(y_train, y_train_pred, average='weighted')  # Asegúrate de ajustar el 'average'
#     recall_train = recall_score(y_train, y_train_pred, average='weighted')
#     f1_train = f1_score(y_train, y_train_pred, average='weighted')

#     # Calcular la precisión (accuracy) para los datos de prueba
#     accuracy_test = accuracy_score(y_test, y_test_pred)
#     precision_test = precision_score(y_test, y_test_pred, average='weighted')
#     recall_test = recall_score(y_test, y_test_pred, average='weighted')
#     f1_test = f1_score(y_test, y_test_pred, average='weighted')

#     # Almacenar los resultados en la lista, usando el nombre del archivo como "dataset"
#     model_results.append(
#     {
#         "dataset_TRAIN": TRAIN_PATHS[index],  
#         "dataset_TEST": TEST_PATHS[index],          
#         "accuracy_train": accuracy_train,  
#         "accuracy_test": accuracy_test,          
#         "precision_train": precision_train, 
#         "precision_test": precision_test,          
#         "recall_train": recall_train,  
#         "recall_test": recall_test,          
#         "f1_train": f1_train,  
#         "f1_test": f1_test,  
#     }
# )

# model

In [9]:
# Ruta base
BASE_PATH = "../data/processed/Excels/"

# Definir las rutas de los archivos de entrenamiento y prueba
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_sel_k4.xlsx"
]

TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_sel_k4.xlsx"
]

# Leer los datasets de entrenamiento
TRAIN_DATASETS = [pd.read_excel(os.path.join(BASE_PATH, path)) for path in TRAIN_PATHS]  # X_train 

# Leer los datasets de prueba
TEST_DATASETS = [pd.read_excel(os.path.join(BASE_PATH, path)) for path in TEST_PATHS]  # X_test

# Leer las etiquetas (target) de entrenamiento y prueba
y_train = pd.read_excel(os.path.join(BASE_PATH, "y_train.xlsx")).values.ravel()
y_test = pd.read_excel(os.path.join(BASE_PATH, "y_test.xlsx")).values.ravel()

all_selected_features = {}
model_results = []
k = 6  # Número de características a seleccionar

for index, dataset in enumerate(TRAIN_DATASETS):
    print(f"Procesando dataset {index + 1}")

    # Selección de características usando SelectKBest con f_classif (para clasificación)
    selector = SelectKBest(f_classif, k=k)
    X_train = selector.fit_transform(dataset, y_train)  # Ajuste y transformación para el conjunto de entrenamiento
    X_test = selector.transform(TEST_DATASETS[index])  # Transformación para el conjunto de prueba

    # Almacenar las características seleccionadas en el diccionario
    selected_features = dataset.columns[selector.get_support()].tolist()  # Obtener los nombres de las características seleccionadas
    all_selected_features[TRAIN_PATHS[index]] = selected_features  # Usar el nombre del archivo como clave

    ### ENTRENAR MODELO ###

    # Crear y entrenar el modelo XGBClassifier
    model = XGBClassifier(random_state=95)  # Ajustar los parámetros según sea necesario
    model.fit(X_train, y_train)

    # Predicciones en los datos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcular la precisión (accuracy) para los datos de entrenamiento
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred, average='weighted')  # Ajustar 'average'
    recall_train = recall_score(y_train, y_train_pred, average='weighted')
    f1_train = f1_score(y_train, y_train_pred, average='weighted')

    # Calcular la precisión (accuracy) para los datos de prueba
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred, average='weighted')
    recall_test = recall_score(y_test, y_test_pred, average='weighted')
    f1_test = f1_score(y_test, y_test_pred, average='weighted')

    # Almacenar los resultados en la lista, usando el nombre del archivo como "dataset"
    model_results.append(
    {
        "dataset_TRAIN": TRAIN_PATHS[index],  
        "dataset_TEST": TEST_PATHS[index],          
        "accuracy_train": accuracy_train,  
        "accuracy_test": accuracy_test,          
        "precision_train": precision_train, 
        "precision_test": precision_test,          
        "recall_train": recall_train,  
        "recall_test": recall_test,          
        "f1_train": f1_train,  
        "f1_test": f1_test,  
    }
)
# Guardar los resultados en un archivo JSON
with open("../models/model_results.json", "w") as json_file:
    json.dump(model_results, json_file, indent=4)

# Guardar las características seleccionadas en un archivo JSON
with open("../models/selected_features.json", "w") as json_file:
    json.dump(all_selected_features, json_file, indent=4)

# Mostrar los resultados
for result in model_results:
    print(f"\nResultados para el dataset: {result['dataset_TRAIN']}")
    print(f"Accuracy Train: {result['accuracy_train']}")
    print(f"Accuracy Test: {result['accuracy_test']}")
    print(f"Precision Train: {result['precision_train']}")
    print(f"Precision Test: {result['precision_test']}")
    print(f"Recall Train: {result['recall_train']}")
    print(f"Recall Test: {result['recall_test']}")
    print(f"F1 Train: {result['f1_train']}")
    print(f"F1 Test: {result['f1_test']}")


Procesando dataset 1
Procesando dataset 2
Procesando dataset 3


/home/vscode/.local/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=6 is greater than n_features=4. All the features will be returned.
  warnings.warn(



Resultados para el dataset: X_train_con_outliers.xlsx
Accuracy Train: 1.0
Accuracy Test: 0.7727272727272727
Precision Train: 1.0
Precision Test: 0.7681086064462481
Recall Train: 1.0
Recall Test: 0.7727272727272727
F1 Train: 1.0
F1 Test: 0.7686942607304208

Resultados para el dataset: X_train_sin_outliers.xlsx
Accuracy Train: 1.0
Accuracy Test: 0.7402597402597403
Precision Train: 1.0
Precision Test: 0.7364035964035964
Recall Train: 1.0
Recall Test: 0.7402597402597403
F1 Train: 1.0
F1 Test: 0.7378112083994437

Resultados para el dataset: X_train_sel_k4.xlsx
Accuracy Train: 1.0
Accuracy Test: 0.7467532467532467
Precision Train: 1.0
Precision Test: 0.7457112288048213
Recall Train: 1.0
Recall Test: 0.7467532467532467
F1 Train: 1.0
F1 Test: 0.746199817757915


In [10]:

# Guardar los resultados del modelo en un DataFrame
df_results = pd.DataFrame(model_results)

# Guardar los resultados en un archivo JSON
result_json_path = os.path.join("../models/model_results_k_{k}.json")
with open(result_json_path, 'w') as json_file:
    json.dump(model_results, json_file, indent=4)

# Guardar las características seleccionadas en un solo archivo JSON
features_json_path = os.path.join(f"../models/selected_features_k_{k}.json")
with open(features_json_path, 'w') as json_file:
    json.dump(all_selected_features, json_file, indent=4)

# Mostrar los resultados
print(f"Resultados de todos los datasets con k: {k}:")
print(df_results.to_string(index=False))

# Selección del mejor dataset basado en la mayor precisión en el conjunto de prueba
best_dataset_index = df_results["accuracy_test"].idxmax()  
best_dataset = df_results.iloc[best_dataset_index]

print("\nEl mejor dataset seleccionado según la mayor precisión en el conjunto de prueba:")
print(best_dataset[["accuracy_train", "accuracy_test"]])

print(f"\nEl mejor dataset de entrenamiento es: {TRAIN_PATHS[best_dataset_index]}")
print(f"El mejor dataset de prueba es: {TEST_PATHS[best_dataset_index]}")


Resultados de todos los datasets con k: 6:
            dataset_TRAIN             dataset_TEST  accuracy_train  accuracy_test  precision_train  precision_test  recall_train  recall_test  f1_train  f1_test
X_train_con_outliers.xlsx X_test_con_outliers.xlsx             1.0       0.772727              1.0        0.768109           1.0     0.772727       1.0 0.768694
X_train_sin_outliers.xlsx X_test_sin_outliers.xlsx             1.0       0.740260              1.0        0.736404           1.0     0.740260       1.0 0.737811
      X_train_sel_k4.xlsx       X_test_sel_k4.xlsx             1.0       0.746753              1.0        0.745711           1.0     0.746753       1.0 0.746200

El mejor dataset seleccionado según la mayor precisión en el conjunto de prueba:
accuracy_train         1.0
accuracy_test     0.772727
Name: 0, dtype: object

El mejor dataset de entrenamiento es: X_train_con_outliers.xlsx
El mejor dataset de prueba es: X_test_con_outliers.xlsx


In [12]:
start_time = time.time()

# Crear y entrenar el modelo XGBoost
model = XGBClassifier(random_state=95)

param_grid = {
    'n_estimators': [100, 200, 300],  # Número de estimadores (árboles)
    'max_depth': [3, 5, 7],  # Profundidad máxima de los árboles
    'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje (eta)
    'subsample': [0.8, 0.9, 1.0],  # Fracción de muestras utilizadas para entrenar cada árbol
    'colsample_bytree': [0.8, 0.9, 1.0],  # Fracción de características utilizadas por cada árbol
    'min_child_weight': [1, 5, 10],  # Mínimo peso de las muestras en cada nodo hijo
    'gamma': [0, 0.1, 0.2],  # Reducción de la pérdida para las particiones de los árboles
    'scale_pos_weight': [1, 5],  # Ajuste de pesos para clases desbalanceadas
    'max_delta_step': [0, 1],  # Paso máximo de delta para estabilizar el entrenamiento
}

# Configuración de GridSearchCV con validación cruzada estratificada
grid = GridSearchCV(
    model, 
    param_grid=param_grid, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),  # 5 pliegues
    n_jobs=-1,  # Utilizar todos los núcleos disponibles
    verbose=0,  # Cambiar verbose a 0 para suprimir la salida detallada
    scoring='accuracy',  # Evaluar el modelo según la precisión
    refit=True,  # Reajustar el modelo con los mejores parámetros encontrados
    return_train_score=True,  # No devolver puntuaciones de entrenamiento
)

# Ajustar el modelo con el conjunto de entrenamiento filtrado
grid.fit(X_train, y_train)
final_model = grid.best_estimator_

# Predicciones en los datos de entrenamiento y prueba
y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)

end_time = time.time()
execution_time = end_time - start_time

data_to_save = {
    "hyperparams": grid.best_params_,
    "train_accuracy": accuracy_score(y_train, y_pred_train),
    "test_accuracy": accuracy_score(y_test, y_pred_test),
    "train_predictions": y_pred_train.tolist(),
    "test_predictions": y_pred_test.tolist(),
}

# Guardamos los resultados en un archivo JSON
with open(os.path.join("../models/hyperparams_XGB.json"), "w") as json_file:
    json.dump(data_to_save, json_file, indent=4)

print(f"Tiempo de ejecución: {execution_time / 60:.0f} minutos")
print(f"Training Accuracy: {accuracy_score(y_train, y_pred_train) * 100:.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_test) * 100:.4f}")


Tiempo de ejecución: 5 minutos
Training Accuracy: 80.6189
Test Accuracy: 77.2727


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

acc = accuracy_score(y_test, y_pred_test)
prec = precision_score(y_test, y_pred_test)
rec = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
cm = confusion_matrix(y_test, y_pred_test)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Accuracy: 0.7727272727272727
Precision: 0.7111111111111111
Recall: 0.5925925925925926
F1 Score: 0.6464646464646465
Confusion Matrix:
 [[87 13]
 [22 32]]
